In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("resume_transformed.csv")

In [3]:
data.head()

,ID,Resume_str,Resume_html,Category,trasnformed_text
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr administrator marketing associate hr admini...
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR,hr specialist u hr operation summary versatile...
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr director summary 20 year experience recruit...
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr specialist summary dedicated driven dynamic...
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr manager skill highlight hr skill hr departm...


In [4]:
data=data.drop(["ID","Resume_str","Resume_html"],axis=1)

In [5]:
len(data["Category"].unique())

24

In [6]:
data.shape

(2484, 2)

In [7]:
data.isna().sum()

Category            0
trasnformed_text    1
dtype: int64

In [8]:
data=data.dropna()

In [9]:
data.shape

(2483, 2)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()

In [11]:
X=tfidf.fit_transform(data["trasnformed_text"]).toarray()

In [12]:
X.shape

(2483, 36270)

In [13]:
# X=X.reshape(X.shape[0],1,X.shape[1])

In [14]:
# X.shape

In [15]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [16]:
y=encoder.fit_transform(data["Category"])

In [17]:
y

array([19, 19, 19, ...,  6,  6,  6])

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=42,test_size=0.2)

In [24]:
from xgboost import XGBClassifier
from sklearn.utils import shuffle
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [48]:
clf_1 = RandomForestClassifier(random_state=1, n_estimators=100, min_samples_split=4, min_samples_leaf=2)
clf_2 = AdaBoostClassifier(random_state=1, n_estimators=100, learning_rate=0.1)
clf_3 = XGBClassifier(n_estimators=196, max_depth=6, learning_rate=0.1)
clf_4 = LogisticRegression(max_iter=1000)
clf_5 = DecisionTreeClassifier()

In [51]:
clfs={
    "clf3":clf_3
    }

In [52]:
from sklearn.metrics import accuracy_score,precision_score
def train(clf_name,clf):
    print(f"working with ",clf_name)
    clf.fit(X_train, y_train)
    y_pred=clf.predict(X_test)
    accuracy= accuracy_score(y_pred, y_test)
    precision= precision_score(y_pred, y_test,average="micro")
    return accuracy,precision

In [53]:



accuracy={}
precisions={}

for clf_name,clf in clfs.items():
    print(f"working with ",clf_name)
    accuracy[clf_name] ,precisions[clf_name]=train(clf_name,clf)
     
    

working with  clf3
working with  clf3


In [41]:
accuracy

{'clf1': 0.6498993963782697,
 'clf2': 0.22132796780684105,
 'clf3': 0.7424547283702213,
 'clf4': 0.6398390342052314,
 'clf5': 0.6116700201207244}

In [44]:
precisions

{'clf1': 0.6498993963782697,
 'clf2': 0.22132796780684105,
 'clf3': 0.7424547283702213,
 'clf4': 0.6398390342052314,
 'clf5': 0.6116700201207244}

In [45]:
import pickle

In [47]:
pickle.dump(clf_3,open("XGBClassifier.pkl","wb"))